Uso:

Función para evaluar y visualizar el rendimiento de modelos de Machine Learning de manera rápida y eficiente. 

Eval Model permite tener una visión clara y detallada de cómo se desempeña un modelo en distintos aspectos, ya sea en tareas de regresión o clasificación.

In [ ]:
def eval_model(target, predictions, problem_type, metrics):
    """
    Evalúa el rendimiento de un modelo de Machine Learning según las métricas especificadas.
    
    Argumentos:
    target: Lista de valores reales del target.
    predictions: Lista de valores predichos por el modelo.
    problem_type (str): Tipo de problema ('regression' o 'classification').
    metrics: Lista de métricas a calcular.
    
    Retorna:
    tuple: Tupla con los valores de las métricas en el orden de aparición en la lista de entrada.
    """
    results = []
    
    # Funciones auxiliares para las métricas de regresión
    def calculate_rmse(target, predictions):
        return np.sqrt(mean_squared_error(target, predictions))
    
    def calculate_mae(target, predictions):
        return mean_absolute_error(target, predictions)
    
    def calculate_mape(target, predictions):
        if np.any(target == 0):
            raise ValueError("MAPE no se puede calcular con valores de target igual a 0")
        return np.mean(np.abs((target - predictions) / target)) * 100
    
    # Funciones auxiliares para métricas de clasificación
    def calculate_accuracy(target, predictions):
        return accuracy_score(target, predictions)
    
    def calculate_precision(target, predictions, average='macro'):
        return precision_score(target, predictions, average=average)
    
    def calculate_recall(target, predictions, average='macro'):
        return recall_score(target, predictions, average=average)
    
    # Procesamiento de métricas según el tipo de problema
    if problem_type == 'regression':
        for metric in metrics:
            if metric == 'RMSE':
                rmse = calculate_rmse(target, predictions)
                print(f"RMSE: {rmse}")
                results.append(rmse)
            elif metric == 'MAE':
                mae = calculate_mae(target, predictions)
                print(f"MAE: {mae}")
                results.append(mae)
            elif metric == 'MAPE':
                try:
                    mape = calculate_mape(target, predictions)
                    print(f"MAPE: {mape}")
                    results.append(mape)
                except ValueError as e:
                    print(e)
            elif metric == 'GRAPH':
                plt.scatter(target, predictions)
                plt.xlabel('Actual')
                plt.ylabel('Predicted')
                plt.title('Actual vs Predicted')
                plt.show()
    
    elif problem_type == 'classification':
        for metric in metrics:
            if metric == 'ACCURACY':
                accuracy = calculate_accuracy(target, predictions)
                print(f"Accuracy: {accuracy}")
                results.append(accuracy)
            elif metric == 'PRECISION':
                precision = calculate_precision(target, predictions)
                print(f"Precision: {precision}")
                results.append(precision)
            elif metric == 'RECALL':
                recall = calculate_recall(target, predictions)
                print(f"Recall: {recall}")
                results.append(recall)
            elif metric == 'CLASS_REPORT':
                report = classification_report(target, predictions)
                print("Classification Report:\n", report)
            elif metric == 'MATRIX':
                matrix = confusion_matrix(target, predictions)
                display = ConfusionMatrixDisplay(confusion_matrix=matrix)
                display.plot()
                plt.show()
            elif metric == 'MATRIX_RECALL':
                display = ConfusionMatrixDisplay.from_predictions(target, predictions, normalize='true')
                display.plot()
                plt.show()
            elif metric == 'MATRIX_PRED':
                display = ConfusionMatrixDisplay.from_predictions(target, predictions, normalize='pred')
                display.plot()
                plt.show()
            elif 'PRECISION_' in metric:
                class_label = metric.split('_')[1]
                precision = precision_score(target, predictions, labels=[class_label], average='macro', zero_division=0)
                if precision == 0:
                    raise ValueError(f"Etiqueta {class_label} no encontrada en target")
                print(f"Precision for class {class_label}: {precision}")
                results.append(precision)
            elif 'RECALL_' in metric:
                class_label = metric.split('_')[1]
                recall = recall_score(target, predictions, labels=[class_label], average='macro', zero_division=0)
                if recall == 0:
                    raise ValueError(f"Etiqueta {class_label} no encontrada en target")
                print(f"Recall for class {class_label}: {recall}")
                results.append(recall)
    
    return tuple(results)

Uso:
Especialmente útil en problemas de clasificación donde se quiere entender que características numéricas pueden influir en la variable objetivo categórica.

Permite enfocarse en aquellas columnas numéricas que tienen una relación estadísticamente significativa con la columna objetivo categórica, facilitando así el proceso de construcción de modelos de clasificación.

In [ ]:

def get_features_num_classification(df, target_col, pvalue=0.05):
    """
    Selecciona columnas numéricas cuyo ANOVA con la columna target supere un nivel de significación.
    
    Argumentos:
    df (pd.DataFrame): DataFrame de entrada.
    target_col (str): Nombre de la columna target.
    pvalue (float): Nivel de significación para el test de ANOVA. Por defecto 0.05.
    
    Retorna:
    list: Lista de columnas numéricas que cumplen con el criterio de ANOVA.
    """
    # Comprobaciones de los argumentos de entrada
    if not isinstance(df, pd.DataFrame):
        print("El argumento 'df' no es un DataFrame.")
        return None
    if target_col not in df.columns:
        print(f"La columna '{target_col}' no existe en el DataFrame.")
        return None
    if not pd.api.types.is_categorical_dtype(df[target_col]) and not pd.api.types.is_object_dtype(df[target_col]):
        print(f"La columna '{target_col}' no es categórica.")
        return None
    if not isinstance(pvalue, float) or not (0 < pvalue < 1):
        print("El argumento 'pvalue' debe ser un float entre 0 y 1.")
        return None
    
    # Convertir target_col a categórica si no lo es
    if not pd.api.types.is_categorical_dtype(df[target_col]):
        df[target_col] = df[target_col].astype('category')
    
    # Filtrar columnas numéricas
    numeric_cols = df.select_dtypes(include=[float, int]).columns.tolist()
    significant_features = []
    
    # Realizar el test ANOVA
    for col in numeric_cols:
        groups = [df[df[target_col] == cat][col].dropna() for cat in df[target_col].cat.categories]
        if all(len(group) > 0 for group in groups):
            _, p_val = f_oneway(*groups)
            if p_val < (1 - pvalue):
                significant_features.append(col)
    
    return significant_features

Uso:

Esta función la utilizamos para visualizar relaciones entre características numéricas y una variable objetivo categórica en un modelo de clasificación. 

 Tiene como objetivo generar gráficos de pares (pairplots) de columnas numéricas en un DataFrame que muestran una relación significativa con una columna objetivo categórica, utilizando el test ANOVA para evaluar la significancia.


In [ ]:

def plot_features_num_classification(df, target_col="", columns=[], pvalue=0.05):
    """
    Genera pairplots de columnas numéricas significativas basadas en ANOVA para clasificación.

    Esta función toma un DataFrame, una columna objetivo categórica y una lista de columnas numéricas, 
    y genera pairplots de las columnas numéricas que tienen una relación significativa con la columna 
    objetivo según el test de ANOVA. Si la lista de columnas está vacía, se consideran todas las columnas 
    numéricas del DataFrame.

    Argumentos:
    df (pd.DataFrame): DataFrame de entrada.
    target_col (str): Nombre de la columna objetivo categórica.
    columns (list): Lista de nombres de columnas numéricas a considerar. Por defecto es una lista vacía.
    pvalue (float): Nivel de significación para el test de ANOVA. Por defecto 0.05.

    Retorna:
    list: Lista de columnas numéricas que cumplen con el criterio de ANOVA.
    """


    # Verificamos si target_col está en el dataframe
    if target_col not in df.columns:
        print("Error: 'target_col' no está en el dataframe.")
        return None
    
    # Verificamos si target_col es categórica
    if not pd.api.types.is_categorical_dtype(df[target_col]) and not pd.api.types.is_object_dtype(df[target_col]):
        print("Error: 'target_col' debe ser una variable categórica.")
        return None
    
    # Si la lista columns está vacía, tomamos todas las columnas numéricas
    if not columns:
        columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Verificamos si las columnas están en el dataframe y son numéricas
    for col in columns:
        if col not in df.columns:
            print(f"Error: La columna '{col}' no está en el dataframe.")
            return None
        if not pd.api.types.is_numeric_dtype(df[col]):
            print(f"Error: La columna '{col}' no es numérica.")
            return None

    # Filtramos las columnas basándonos en el test de ANOVA
    columnas_significativas = []
    categorias_target = df[target_col].dropna().unique()
    if len(categorias_target) < 2:
        print("Error: 'target_col' debe tener al menos dos valores únicos.")
        return None

    for col in columns:
        grupos = [df[df[target_col] == categoria][col].dropna() for categoria in categorias_target]
        if len(grupos) > 1 and all(len(grupo) > 1 for grupo in grupos):
            estadistico_f, valor_p = f_oneway(*grupos)
            if valor_p < pvalue:
                columnas_significativas.append(col)

    if not columnas_significativas:
        print("No hay columnas que pasen el test de ANOVA.")
        return []

    # Definimos el número de valores por gráfico para las categorías del target
    max_valores_target_por_grafico = 5
    max_columnas_por_grafico = 5

    divisiones_target = [categorias_target[i:i + max_valores_target_por_grafico] for i in range(0, len(categorias_target), max_valores_target_por_grafico)]
    divisiones_columnas = [columnas_significativas[i:i + max_columnas_por_grafico] for i in range(0, len(columnas_significativas), max_columnas_por_grafico)]

    # Creamos los pairplots
    for division_target in divisiones_target:
        for division_columnas in divisiones_columnas:
            subset_df = df[df[target_col].isin(division_target)]
            columnas_para_graficar = division_columnas + [target_col]
            sns.pairplot(subset_df[columnas_para_graficar], hue=target_col)
            plt.show()

    return columnas_significativas


Uso:

Selección de características categóricas en problemas de clasificación, identificando aquellas que tienen una relación significativa con la variable objetivo. 

La función combina la validación de entradas, codificación de datos y cálculos estadísticos para proporcionar una lista de características significativas que pueden ser utilizadas en la construcción de modelos de clasificación y tambien puede ayudar a reducir el número de características mejorando el rendimiento del modelo

In [ ]:
def get_features_cat_classification_2(df, target_col, normalize=False, mi_threshold=0.0):
    """
    Selecciona columnas categóricas cuyo valor de mutual information con la columna target supere un umbral.

    Argumentos:
    df (pd.DataFrame): DataFrame de entrada.
    target_col (str): Nombre de la columna target.
    normalize (bool): Si es True, se normaliza el valor de mutual information. Por defecto es False.
    mi_threshold (float): Umbral para el valor de mutual information. Por defecto es 0.

    Retorna:
    list: Lista de columnas categóricas que cumplen con el criterio de mutual information.
    """
    
    # Verificamos si target_col está en el dataframe
    if target_col not in df.columns:
        print("Error: 'target_col' no está en el dataframe.")
        return None
    
    # Verificamos si target_col es categórica o numérica discreta con baja cardinalidad
    if not (pd.api.types.is_categorical_dtype(df[target_col]) or pd.api.types.is_object_dtype(df[target_col]) or 
            (pd.api.types.is_numeric_dtype(df[target_col]) and df[target_col].nunique() < 20)):
        print("Error: 'target_col' debe ser una variable categórica o numérica discreta con baja cardinalidad.")
        return None
    
    # Verificamos si mi_threshold es un float válido
    if not isinstance(mi_threshold, float):
        print("Error: 'mi_threshold' debe ser un valor float.")
        return None

    # Si normalize es True, verificamos que mi_threshold esté entre 0 y 1
    if normalize and not (0 <= mi_threshold <= 1):
        print("Error: 'mi_threshold' debe estar entre 0 y 1 cuando 'normalize' es True.")
        return None
    
    # Convertir columnas categóricas a códigos numéricos
    le = LabelEncoder()
    df_categorical = df.select_dtypes(include=['category', 'object'])
    df_categorical_encoded = df_categorical.apply(lambda x: le.fit_transform(x))
    
    # Unir las columnas categóricas convertidas con las columnas numéricas originales
    df_encoded = pd.concat([df.drop(df_categorical.columns, axis=1), df_categorical_encoded], axis=1)
    
    # Calculamos la mutual information para las columnas categóricas convertidas
    cols_categoricas_encoded = df_categorical_encoded.columns.tolist()
    mi_scores = mutual_info_classif(df_encoded[cols_categoricas_encoded], df_encoded[target_col], discrete_features=True)

    if normalize:
        # Normalizamos los valores de mutual information
        mi_scores /= mi_scores.sum()
    
    # Seleccionamos las columnas que cumplen con el umbral
    columnas_significativas = [col for col, mi in zip(cols_categoricas_encoded, mi_scores) if mi >= mi_threshold]

    return columnas_significativas

### USO:

La función plot_features_cat_classification genera gráficos de distribución para columnas categóricas que tienen una relación significativa con una columna objetivo (target) basada en el valor de información mutua (mutual information). Este tipo de visualización es útil para entender cómo las categorías de las características categóricas están distribuidas en relación con la variable objetivo.

En resumen su objetivo es crear una visualización de las categorías con relación a la variable objetivo. Esto facilita la interpretación de las variables categóricas en un modelo de clasificación.




In [ ]:

def plot_features_cat_classification(df, target_col="", columns=[], mi_threshold=0.0, normalize=False):
    """
    Genera gráficos de distribución para columnas categóricas significativas basadas en mutual information.

    Esta función toma un DataFrame, una columna objetivo categórica y una lista de columnas categóricas, 
    y genera gráficos de distribución de las columnas categóricas que tienen una relación significativa con la columna 
    objetivo según el valor de mutual information. Si la lista de columnas está vacía, se consideran todas las columnas 
    categóricas del DataFrame.

    Argumentos:
    df (pd.DataFrame): DataFrame de entrada.
    target_col (str): Nombre de la columna objetivo categórica.
    columns (list): Lista de nombres de columnas categóricas a considerar. Por defecto es una lista vacía.
    mi_threshold (float): Umbral para el valor de mutual information. Por defecto es 0.0.
    normalize (bool): Si es True, se normaliza el valor de mutual information. Por defecto es False.

    Retorna:
    list: Lista de columnas categóricas que cumplen con el criterio de mutual information.
    """

    # Verificamos si target_col está en el dataframe
    if target_col not in df.columns:
        print("Error: 'target_col' no está en el dataframe.")
        return None
    
    # Verificamos si target_col es categórica o numérica discreta con baja cardinalidad
    if not (pd.api.types.is_categorical_dtype(df[target_col]) or pd.api.types.is_object_dtype(df[target_col]) or 
            (pd.api.types.is_numeric_dtype(df[target_col]) and df[target_col].nunique() < 20)):
        print("Error: 'target_col' debe ser una variable categórica o numérica discreta con baja cardinalidad.")
        return None
    
    # Verificamos si mi_threshold es un float válido
    if not isinstance(mi_threshold, float):
        print("Error: 'mi_threshold' debe ser un valor float.")
        return None

    # Si normalize es True, verificamos que mi_threshold esté entre 0 y 1
    if normalize and not (0 <= mi_threshold <= 1):
        print("Error: 'mi_threshold' debe estar entre 0 y 1 cuando 'normalize' es True.")
        return None
    
    # Si la lista columns está vacía, tomamos todas las columnas categóricas
    if not columns:
        columns = df.select_dtypes(include=['category', 'object']).columns.tolist()
    
    # Verificamos si las columnas están en el dataframe y son categóricas
    for col in columns:
        if col not in df.columns:
            print(f"Error: La columna '{col}' no está en el dataframe.")
            return None
        if not (pd.api.types.is_categorical_dtype(df[col]) or pd.api.types.is_object_dtype(df[col])):
            print(f"Error: La columna '{col}' no es categórica.")
            return None

    # Calculamos la mutual information para las columnas categóricas
    mi_scores = mutual_info_classif(df[columns], df[target_col], discrete_features=True)

    if normalize:
        # Normalizamos los valores de mutual information
        mi_scores /= mi_scores.sum()
    
    # Seleccionamos las columnas que cumplen con el umbral
    columnas_significativas = [col for col, mi in zip(columns, mi_scores) if mi >= mi_threshold]

    if not columnas_significativas:
        print("No hay columnas que pasen el umbral de mutual information.")
        return []

    # Graficamos la distribución de etiquetas para cada columna significativa
    for col in columnas_significativas:
        plt.figure(figsize=(10, 6))
        sns.countplot(data=df, x=col, hue=target_col)
        plt.title(f'Distribución de {col} respecto a {target_col}')
        plt.xlabel(col)
        plt.ylabel('Frecuencia')
        plt.xticks(rotation=45)
        plt.legend(title=target_col)
        plt.show()

    return columnas_significativas
